<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/FAISS_FAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nivia-smi

/bin/bash: nivia-smi: command not found


In [2]:
path = '/content/drive/My Drive/Colab Notebooks/BERT/Doc2Query'

In [3]:
!pip install -q transformers
!apt install -q libomp-dev
!python -m pip -q install --upgrade faiss faiss-gpu

     |████████████████████████████████| 890kB 2.6MB/s 
     |████████████████████████████████| 890kB 14.2MB/s 
     |████████████████████████████████| 3.0MB 17.0MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 2s (120 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 144676 files and directories currently installed.)
Preparing to unpack 

In [10]:
# Pacotes Básicos
import time 
import numpy as np 
import pandas as pd

# PyTorch
import torch
from torch import Tensor

# Transformers
import transformers

# Faiss
import faiss

# Modelo é o BERTimbau
MODEL = 'neuralmind/bert-base-portuguese-cased'

# Carrega o Modelo
bertimbau = transformers.BertModel.from_pretrained(MODEL)

# Tokenizador do BERTimbau
tokenizer = transformers.BertTokenizer.from_pretrained(MODEL)

# Envia o modelo para GPU
bertimbau.cuda(0)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [132]:
df_faq = pd.read_csv('/content/drive/My Drive/Colab Notebooks/BERT/Doc2Query/FAQ_test_DF.csv', index_col=0)
df_faq.head(3)

,qid,docid,question,doc
0,458,683,Como habilito ou cancelo o boleto eletrônico dda?,Você habilita ou cancela aqui mesmo no celular...
1,323,918,Como consulto o código de barras da minha fatura?,O número do código de barras da fatura é o mes...
2,782,738,Como criar boleto no iti?,"Na tela inicial do app, vá em ""colocar dinheir..."


In [133]:
def mapping_index(dataframe):
  # Dict com k: index --> v: docid: doc
  index_map = {}

  # Para cada tupla (index, cols)....
  for tup in dataframe.itertuples():
    # Anexa o index no Dict
    index_map[tup[0]] = {
                        'docid': tup[2],
                        'doc': tup[4],
                        }
  return index_map

index_map = mapping_index(df_faq)
index_map.items()

dict_items([(0, {'docid': 683, 'doc': 'Você habilita ou cancela aqui mesmo no celular.  basta acessar o menu operações e clicar no botão "pagamentos". lá, desça a tela e escolha a opção "habilitar boleto eletrônico (dda)" ou "cancelar boleto eletrônico (dda)". depois é só seguir o passo a passo e confirmar.'}), (1, {'docid': 918, 'doc': 'O número do código de barras da fatura é o mesmo todos os meses, nunca muda e você consegue acessar aqui mesmo pelo app. pra isso, toque em “home”, depois em “fatura” e aí é só selecionar a opção "pagar" ou "abrir 2ª via em pdf" para pegar o código. ;) código para pagar fatura'}), (2, {'docid': 738, 'doc': 'Na tela inicial do app, vá em "colocar dinheiro" e escolha "boleto". depois, é só digitar o valor que quer colocar no iti, tocar em "criar boleto", e pronto!  você pode pagar em qualquer banco, lotérica ou pela internet enviando o código de barras para seu e-mail  ah, vale lembrar que o boleto é emitido em seu nome e deve ser pago somente por você  

In [153]:
def compute_embeddings(data, K=384):
    # BERTimbau em modo avaliação
    bertimbau.eval()
    
    # Lista que armazena os embeddings
    embeddings = []

    # Para cada doc em data...
    for i, doc in enumerate(data):
        
        # Tokeniza o exemplo 
        input_ids = tokenizer.encode(
            doc, # texto 
            add_special_tokens=True,  # add [CLS] e [SEP]
            max_length=K, # tamanno max. da sequencia
            truncation=True, # truncamento
            return_tensors='pt' # retorna tensores
            ).to('cuda') 

        # Não calcula o gradiente
        with torch.no_grad():
            # Calcula a predição do modelo
            ypred = bertimbau(input_ids)[0]

        # [:, 1:-1] para não pegar o token [CLS] e [SEP], calcula a média dim=1
        ypred_mean = ypred[:, 1:-1].mean(dim=1)[0].cpu().detach().numpy()
        # ypred_mean = ypred.mean(dim=1)[0].cpu().detach().numpy()
        
        # Anexa a predição na lista
        embeddings.append(ypred_mean)   

    return  np.array(embeddings) # retorna em formato numpy

In [154]:
# Calcula os embeddings dos docs
docs_embs = compute_embeddings(df_faq.doc.to_list())
docs_embs.shape

(1062, 768)

In [155]:
# Função que calcula a similaridade cosseno
def pytorch_cos_sim_np(a: Tensor, b: Tensor):

    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    cos = torch.einsum('ab, cb -> ac', a_norm, b_norm)
    cos = cos.detach().cpu().numpy()
    return np.float(cos.flatten())

In [156]:
# Seleção de um exemplo
df_faq.groupby(['qid']).agg(lambda x: tuple(x))\
.applymap(list).reset_index()[['qid', 'docid']][:20]

E = 1
q = [df_faq[df_faq.qid==E].question.to_list()[0]]

df_faq[df_faq.qid==E]

,qid,docid,question,doc
666,1,203,"Fiz o parcelamento, mas não consigo consultar....",Você só consegue consultar o seu parcelamento ...


In [168]:
def index_to_title(indexes):
    for i, idx in enumerate(indexes):
        print(f'{i:4}. {index_map[idx]["docid"]:>4} -- {index_map[idx]["doc"]}')

def do_consine_search(embeddings, query_text, top_k):

    n_embeddings = embeddings.shape[0]

    embedding_q = compute_embeddings(query_text)[0]
    
    # lets do the search and time the process
    st = time.time()
    distances = []
    for em in embeddings:
        em = np.expand_dims(em, axis = 0)
        distances.append(pytorch_cos_sim_np(em, embedding_q))
        
    top_k_arguments = np.argsort(np.array(distances))[::-1][:top_k]
    et = time.time()
    
    return et - st, top_k_arguments
    
time_cosine, indexes_top = do_consine_search(docs_embs, q, 20)
print(f'\tCosine search time: {time_cosine:.6} seconds')
print(f'\tQuestion: {q[0]}\n')
index_to_title(indexes_top)

	Cosine search time: 0.106288 seconds
	Question: Posso cadastrar a biometria em qualquer agência?

   0.  439 -- A biometria fica disponível imediatamente após o cadastramento.
   1.  971 -- Com o cartão em mãos, você pode alterar a senha em qualquer caixa eletrônico itaú acessando a função troca de senha.
   2.  969 -- Nesse caso será necessário entrar em contato com a central de atendimento, atualizar seu endereço e realizar uma nova solicitação de emissão do cartão para o endereço correto.
   3.   63 -- O cadastro não é obrigatório, porém algumas transações só podem ser feitas com o uso da biometria.
   4.  808 -- Não, o cadastro do seu cartão no paypal é totalmente gratuito e permite apenas que você não precise colocar os dados do seu cartão a cada compra.
   5.  658 -- Não, o cadastro do seu cartão no paypal é totalmente gratuito e permite apenas que vc não precise colocar os dados do seu cartão a cada compra.
   6.   26 -- Se sua senha foi bloqueada, você precisa esperar 22 dias 

In [167]:
E = 6
q = [df_faq[df_faq.qid==E].question.to_list()[0]]

df_faq[df_faq.qid==E]

,qid,docid,question,doc
826,6,845,Posso cadastrar a biometria em qualquer agência?,"Sim, você pode cadastrar a biometria em qualqu..."


In [163]:
n_dimensions = docs_embs.shape[1] # Number of dimensions (768)
fastIndex = faiss.IndexFlatL2(n_dimensions) # We will create an index of type FlatL2, there are many kinds of indexes, you can look at it in their repo.
fastIndex.add(docs_embs.astype('float32')) # Add the embedding vector to faiss index, it should of dtype 'float32'

In [166]:
def do_faiss_lookup(fastIndex, query_text, top_k):
    n_embeddings = docs_embs.shape[0]
    embedding_q = compute_embeddings(query_text)
    
    #let it be float32
    embedding_q = embedding_q.astype('float32')
    
    #perform the search
    st = time.time()

    # it returns matched vectors and thier respective indexes, we are interested only in indexes
    matched_em, matched_indexes = fastIndex.search(embedding_q, top_k) 
    
    #indexes are already sorted wrt to closest match
    et = time.time()
    
    return et - st, matched_indexes[0]


E = 6
q = [df_faq[df_faq.qid==E].question.to_list()[0]]

print(df_faq[df_faq.qid==E].docid.values)    

time_faiss, indexes_top_faiss = do_faiss_lookup(fastIndex, q, 20)
print(f'\tFaiss time: {time_faiss:.6} seconds')
print(f'\tQuestion: {q[0]}\n')
index_to_title(indexes_top_faiss) 

[845]
	Faiss time: 0.00100803 seconds
	Question: Posso cadastrar a biometria em qualquer agência?

   0.  439 -- A biometria fica disponível imediatamente após o cadastramento.
   1.  971 -- Com o cartão em mãos, você pode alterar a senha em qualquer caixa eletrônico itaú acessando a função troca de senha.
   2.   26 -- Se sua senha foi bloqueada, você precisa esperar 22 dias úteis (sem contar finais de semana e feriados) para iniciar a solicitação de abertura de conta pelo aplicativo de novo. se preferir, você pode abrir sua conta em qualquer agência itaú.
   3.  808 -- Não, o cadastro do seu cartão no paypal é totalmente gratuito e permite apenas que você não precise colocar os dados do seu cartão a cada compra.
   4.  658 -- Não, o cadastro do seu cartão no paypal é totalmente gratuito e permite apenas que vc não precise colocar os dados do seu cartão a cada compra.
   5.  205 -- Você pode sim. todos os caixas do banco24horas permitem usar a biometria.
   6.  324 -- Nesta funcionali

In [169]:
df_faq[df_faq.qid==E]

,qid,docid,question,doc
826,6,845,Posso cadastrar a biometria em qualquer agência?,"Sim, você pode cadastrar a biometria em qualqu..."


In [170]:
print(f'Cosine-distance metric lookup (Brute-force method:) {time_cosine: .6}')
print(f'Faiss index: {time_faiss: .6}')

Cosine-distance metric lookup (Brute-force method:)  0.106288
Faiss index:  0.00100803
